# Part 3. Model Training & Evaluation - biGRU

## Import Embedding Matrix

In [15]:
import json
from pathlib import Path

import numpy as np

embedding_path = Path("models/embedding_matrix_oov.npy")
index_from_word_path = Path("models/index_from_word_oov.json")

embedding_matrix = np.load(embedding_path)
with index_from_word_path.open() as f:
    index_from_word = json.load(f)

## Prepare Dataset

In [16]:
from utils.text import tokenize
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
train_dataset = tokenize(dataset["train"])
val_dataset = tokenize(dataset["validation"])
test_dataset = tokenize(dataset["test"])

In [17]:
from utils.text import token_to_index

train_dataset = token_to_index(dataset=train_dataset, index_from_word=index_from_word)
val_dataset = token_to_index(dataset=val_dataset, index_from_word=index_from_word)
test_dataset = token_to_index(dataset=test_dataset, index_from_word=index_from_word)

train_dataset

Dataset({
    features: ['text', 'label', 'tokens', 'original_len', 'indexes'],
    num_rows: 8530
})

In [18]:
train_dataset = train_dataset.select_columns(["label", "original_len", "indexes"])
val_dataset = val_dataset.select_columns(["label", "original_len", "indexes"])
test_dataset = test_dataset.select_columns(["label", "original_len", "indexes"])

In [19]:
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

## Train RNN - biGRU Model

Hieuristic search with Optuna

In [ ]:
import optuna
from utils.train import train_rnn_model_with_parameters

_N_TRIALS = 150
SEARCH_SPACE = {
    "batch_size": [32, 128, 512, 1024, 2048],
    "learning_rate": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    "optimizer_name": ["Adagrad", "RMSprop", "Adam", "SGD"],
    # biGRU Model Parameters
    "hidden_dim": [256, 128, 64, 32],
    "num_layers": [1, 2, 4],
    "sentence_representation_type": ["last", "average", "max"],
}

def objective(trial):
    hidden_dim = trial.suggest_categorical("hidden_dim", SEARCH_SPACE["hidden_dim"])
    num_layers = trial.suggest_int("num_layers", min(SEARCH_SPACE["num_layers"]), max(SEARCH_SPACE["num_layers"]))
    optimizer_name = trial.suggest_categorical("optimizer_name", SEARCH_SPACE["optimizer_name"])
    batch_size = trial.suggest_categorical("batch_size", SEARCH_SPACE["batch_size"])
    learning_rate = trial.suggest_categorical("learning_rate", SEARCH_SPACE["learning_rate"])
    sentence_representation_type = trial.suggest_categorical("sentence_representation_type", SEARCH_SPACE["sentence_representation_type"])

    
    log_message = f"---------- batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers}; sentence_representation_{sentence_representation_type}; bidirectional_{bidirectional} ----------"
    print(log_message)

    val_acc = train_rnn_model_with_parameters(
        embedding_matrix=embedding_matrix,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        batch_size=batch_size,
        learning_rate=learning_rate,
        optimizer_name=optimizer_name,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        sentence_representation_type=sentence_representation_type,
        show_progress=True,
        log_dir="bigru",
        rnn_type="GRU",
        bidirectional=True,
        freeze_embedding=False,
    )
    
    return val_acc

# Set up the Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=_N_TRIALS)

## Train Result Analysis

In [31]:
from utils.analytics import load_tensorboard_logs

train_results_df = load_tensorboard_logs(log_dir="tb_logs/bigru")
train_results_df = train_results_df.sort_values(
    by=["val_acc"], ascending=False
).reset_index(drop=True)

train_results_df.head(20)

,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.786116,2048,32,0.0010,Adam,0.102713,0.967031,2,last,False,9.0,0.475645,events.out.tfevents.1731150840.yuriarch.240111.0
1,0.782364,2048,32,0.0001,RMSprop,0.246216,0.931981,3,max,False,23.0,0.485582,events.out.tfevents.1731153742.yuriarch.240111.6
2,0.775797,2048,128,0.0001,RMSprop,0.296620,0.877599,2,last,False,14.0,0.481261,events.out.tfevents.1731145891.yuriarch.177922.12
3,0.774859,2048,128,0.0010,RMSprop,0.159597,0.944137,3,last,False,7.0,0.491637,events.out.tfevents.1731145419.yuriarch.177922.10
4,0.774859,2048,32,0.0001,Adam,0.259144,0.932221,4,max,False,40.0,0.498396,events.out.tfevents.1731151155.yuriarch.240111.3
5,0.771107,2048,128,0.0100,Adagrad,0.115080,0.958644,2,last,False,7.0,0.505339,events.out.tfevents.1731151040.yuriarch.240111.2
6,0.767355,2048,256,0.0001,Adagrad,0.451762,0.760647,2,last,False,82.0,0.485582,events.out.tfevents.1731146101.yuriarch.177922.13
7,0.763602,2048,256,0.0001,RMSprop,0.437234,0.793838,2,last,False,5.0,0.496453,events.out.tfevents.1731149937.yuriarch.177922.15
8,0.763602,2048,64,0.0001,RMSprop,0.353918,0.863093,1,last,False,16.0,0.481630,events.out.tfevents.1731153664.yuriarch.240111.5
9,0.761726,2048,256,0.0001,Adagrad,0.502916,0.749228,4,last,False,40.0,0.495599,events.out.tfevents.1731151399.yuriarch.240111.4


(b) Configuration for best trial result

In [32]:
best_rnn_model_configuration = train_results_df.head(1)
best_rnn_model_configuration

,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.786116,2048,32,0.001,Adam,0.102713,0.967031,2,last,False,9.0,0.475645,events.out.tfevents.1731150840.yuriarch.240111.0


## Performance on Test Dataset

In [ ]:
from utils.analytics import test_top_n_models
from models.RNN import RNNClassifier

test_results_df = test_top_n_models(train_results_df, RNNClassifier, test_dataset, n=5)

In [36]:
test_results_df

,test_acc,test_loss,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.798311,0.440092,0.786116,2048,32,0.0010,Adam,0.102713,0.967031,2,last,False,9.0,0.475645,events.out.tfevents.1731150840.yuriarch.240111.0
1,0.811445,0.435418,0.782364,2048,32,0.0001,RMSprop,0.246216,0.931981,3,max,False,23.0,0.485582,events.out.tfevents.1731153742.yuriarch.240111.6
2,0.804878,0.440901,0.775797,2048,128,0.0001,RMSprop,0.296620,0.877599,2,last,False,14.0,0.481261,events.out.tfevents.1731145891.yuriarch.177922.12
3,0.77955,0.442962,0.774859,2048,128,0.0010,RMSprop,0.159597,0.944137,3,last,False,7.0,0.491637,events.out.tfevents.1731145419.yuriarch.177922.10
4,0.780488,0.480579,0.774859,2048,32,0.0001,Adam,0.259144,0.932221,4,max,False,40.0,0.498396,events.out.tfevents.1731151155.yuriarch.240111.3
